## 3.5 出力層の設計

活性関数は問題に応じて以下の関数を使うことが多い。

- 分類問題: ソフトマックス関数
- 回帰問題: 恒等写像

### 3.5.1 恒等写像とソフトマックス関数

#### 恒等写像

入力をそのまま出力する。
\begin{align*}
{\rm id}(x)=x
\end{align*}

#### ソフトマックス関数

\begin{align*}
y = \dfrac{{\rm exp}(a)}{\sum^n_{i=1}{\rm exp}(a_i)}
\end{align*}

In [1]:
import numpy as np
a = np.array([0.3, 2.9, 4.5])
# 指数(分子)
exp_a = np.exp(a)
print(exp_a)

[  1.34985881  18.17414537  90.0171313 ]


In [2]:
# 指数の総和(分母)
sum_exp_a = np.sum(exp_a)
print(sum_exp_a)

109.541135478


In [3]:
# ソフトマックス関数値
y = exp_a / sum_exp_a
print(y)

[ 0.01232285  0.1659116   0.82176555]


In [4]:
# ソフトマックス関数
def softmax(a):
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    return exp_a / sum_exp_a

a = np.array([0.3, 2.9, 4.5])
print(softmax(a))

[ 0.01232285  0.1659116   0.82176555]


### 3.5.2 ソフトマックス関数実装上の注意

指数を扱うのですぐにオーバーフローしてしまう。

#### 改良ソフトマックス関数

\begin{align*}
y = \dfrac{{\rm exp}(a)}{\sum_{i=1}^n{\rm exp}(a_i)} 
&=& \dfrac{C {\rm exp}(a)}{C\sum_{i=1}^n{\rm exp}(a_i)} \\
&=& \dfrac{{\rm exp}(a + \log{C})}{\sum_{i=1}^n{\rm exp}(a_i + \log{C})} \\
&=& \dfrac{C {\rm exp}\left(a + C' \right)}{\sum_{i=1}^n{\rm exp} \left(a_i + C' \right)}
\end{align*}

すなわち, ソフトマックス関数は指数関数の引数に対して任意の定数を足し引きしても結果は等しくなる。

上記の性質を活かし, オーバーフロー対策として入力信号の中の最大値を減ずるような実装をするのが一般的。

In [5]:
# オーバーフローしてしまうケース
softmax(np.array([1010, 1000, 990]))

/home/nagomiso/.anyenv/envs/pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: overflow encountered in exp
  app.launch_new_instance()
/home/nagomiso/.anyenv/envs/pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in true_divide


array([ nan,  nan,  nan])

In [6]:
def enhanced_softmax(a):
    c = np.max(a)
    # オーバーフロー対策
    exp_a = np.exp(a - c)
    sum_exp_a = np.sum(exp_a)
    return exp_a / sum_exp_a

# ちゃんと計算できる
enhanced_softmax(np.array([1010, 1000, 990]))

array([  9.99954600e-01,   4.53978686e-05,   2.06106005e-09])

In [7]:
# 値は改善前と同じ
a = np.array([0.3, 2.9, 4.5])
print(softmax(a))
print(enhanced_softmax(a))

[ 0.01232285  0.1659116   0.82176555]
[ 0.01232285  0.1659116   0.82176555]


### 3.5.3 ソフトマックス関数の特徴

\begin{align*}
{\rm softmax}: \mathbb{R}\rightarrow [0, 1]
\end{align*}

$[0, 1]$ にマッピングできるかつ, ソフトマックス関数の出力の総和は $1$ になる。
ゆえに, ソフトマックス関数の出力は確率として解釈できる。

In [8]:
a = np.array([0.3, 2.9, 4.0])
y = softmax(a)

# [0,1] にマッピングできる
y

array([ 0.01821127,  0.24519181,  0.73659691])

In [9]:
# 総和が 1 になる
np.sum(y)

1.0

各値がクラスに属する確率を意味すると解釈できる。

指数関数は単調関数なので, 入力信号同士の大小関係は変化しない。
ゆえに計算コスト削減のためにクラス分類における出力層では, ソフトマックス関数を適用しないことが多い。

### 3.5.4 出力層のニューロン数

クラス分類ならば出力層のニューロン数はクラス数分にすることが多い。